In [7]:
# To collect the links:
import os
import sys
from bs4 import BeautifulSoup
import urllib.request as ul
import requests as r
import csv
import pandas as pd
import re
from selenium import webdriver
from parsel import Selector 
from collections import defaultdict
import time


house_links = [] 
for i in range(40,65):
 
    driver = webdriver.Chrome()
    url = ('https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=' + str(i) + '&orderBy=relevance')
    driver.get(url)
  
    print("Search Criteria :"  + url)
    soup = BeautifulSoup(driver.page_source, "html.parser")   
   
    
    rawhouselinks = soup.find_all("a", {"class":"card__title-link"}) 

    for item in rawhouselinks:
        try:

            link = item.get('href')         
            house_links.append(link)

        except AttributeError:
            rawhouselinks != 'data'        
    
 


Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=40&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=41&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=42&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=43&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=44&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=45&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=46&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=47&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=48&orderBy=relevance
Search Criteria :https://www.immoweb.be/en/search/house/for-sale

In [8]:
# To clean and sort the data from the links:
import requests as r
import os
import sys
from bs4 import BeautifulSoup
import urllib.request as ul
import requests as r
import csv
import re
from selenium import webdriver
from parsel import Selector 
from collections import defaultdict
import time
 

houseinfo = [] 
for item in house_links:
  try:   
    driver = webdriver.Chrome()
    driver.get(item)
    req = r.get(item)
    soup = BeautifulSoup(driver.page_source, "html.parser")   
   
    
    table = soup.find_all("table", {"class":"classified-table"})    
    rows=[]
    for t in table:     
      trs = t.findAll("tr")
      for tr in trs:
        rows.append(tr)
    
    
    id = item.rsplit('/')[-1]
    zipcode = item.rsplit('/')[-2]
    locality = item.rsplit('/')[-3]
   
    url = str(item)   
    if url.find('apartment')>0:
      prop_type = 'Apartment'
    elif url.find('house')>0:
      prop_type = 'House'
    elif url.find('mansion')>0:
      prop_type = 'Mansion'
    elif url.find('chalet')>0:
      prop_type = 'Chalet'
    else:
      prop_type = 'NaN'


    data={}

    fieldofinterest = { 'ID':id, 
                       'locality': locality,
                       'zipcode' : zipcode,
                       'Building condition': 'NaN',
                       'Price': 0,
                       'Kitchen type':'None',
                       'Living area': 0,
                       'Furnished': 'None',
                       'Surface of the plot': 0,
                       'Number of frontages': 0,
                       'Swimming pool': 'None',
                       'Bedrooms': 0,
                       'Garden' : 'None',
                       'Garden surface' : 0,
                       'Terrace':'None',
                       'Terrace surface': 0,
                       'Type of property' : prop_type                       
                       }

    for item in rows:  
      
      try:      
        if len(item.findAll("th", {"class":"classified-table__header"})) > 0: #   check if the html found then only read the Text 
          key = str(item.find("th", {"class":"classified-table__header"}).getText()).replace("  "," ").strip(" ,\r\n\t")          
          v1 = str(item.find("td", {"class":"classified-table__data"}).getText()).replace("  "," ").strip(" ,\r\n\t")
          value = " ".join(v1.split())  
          #print(str(key)+ ":::" + str(value))
          if(fieldofinterest.__contains__(key)): 
            fieldofinterest[key] =  value
            # All custom logic goes here : 
            if key == 'Garden surface':
               fieldofinterest["Garden"] = 'Yes'         
            if key == 'Terrace surface':
               fieldofinterest["Terrace"] = 'Yes'   
            if key == 'Price' :           
               #p= value.replace(',','').split() 
               valuewithoutcomma= value.replace(',','')
               p1= re.findall("\d+", valuewithoutcomma)    
               fieldofinterest['Price'] = int(p1[0])


              

           
      except:
          print("Warning 1 : Trying to read some out of scope section")  
   
    houseinfo.append(fieldofinterest)
  except:
        print("Warning 2 : Trying to read some out of scope section")  
         




In [ ]:
#To create CSV file 
import pandas as pd

#print(houseinfo)  

df = pd.DataFrame.from_dict(houseinfo) 
df.to_csv (r'immo_house_40_65.csv',index = False, header=True)